# Synthetic Bioinformatics Data Analysis Using GPU-Accelerated Methods

This notebook explores the performance of various bioinformatics operations using both **CPU-based** and **GPU-accelerated** methods. We generate synthetic data to simulate genotype and phenotype datasets and perform tasks such as **GWAS** and **dimensionality reduction (t-SNE, UMAP and PCA)**.

We use **NVIDIA RAPIDS** to leverage GPU acceleration for the analysis and compare its performance to traditional CPU-based methods.

## Steps:
1. Install required packages.
2. Generate synthetic data
3. Compare performance of GPU-based RAPIDS vs CPU-based libraries for different tasks.
4. Analyze and visualize the results.


## Install packages

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py


In [ ]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8

!pip install psutil gputil

In [ ]:
!pip install umap-learn

## Synthetic Data Generation

In this section, we generate synthetic **genotype** and **phenotype** data to simulate a bioinformatics dataset. The genotype data consists of **SNPs** (genetic variants), while the phenotype data corresponds to binary traits.

- **Genotype data**: We simulate a matrix where rows represent individuals (samples) and columns represent SNPs.
- **Phenotype data**: Binary labels representing a hypothetical trait such as disease presence or hearing status.

This synthetic data will be used for various bioinformatics operations throughout the notebook.


In [ ]:
import numpy as np
import pandas as pd
import cupy as cp


# Function to generate synthetic genotype data
def generate_synthetic_data(n_samples, n_snps):
    np.random.seed(42)
    genotype_matrix_cpu = np.random.randint(0, 3, size=(n_samples, n_snps))
    phenotype_data_cpu = np.random.randint(0, 2, size=n_samples)
    genotype_matrix_gpu = cp.array(genotype_matrix_cpu)
    phenotype_data_gpu = cp.array(phenotype_data_cpu)
    return genotype_matrix_cpu, phenotype_data_cpu, genotype_matrix_gpu, phenotype_data_gpu


# # Create datasets of varying sizes
# datasets = {}
# for size in dataset_sizes:
#     datasets[size] = generate_synthetic_data(n_samples=1000, n_snps=size)

## Set up CPU and GPU Monitoring

This section defines utility functions to monitor system resources during the benchmarking process. Specifically, the functions retrieve:
1. **CPU Usage**: The percentage of CPU usage and the amount of memory used by the system.
2. **GPU Utilization**: The percentage of GPU usage and memory utilized by the GPU.

These metrics help track the resource consumption during operations, which is particularly important when comparing CPU-based (pandas, sklearn) and GPU-based (cuDF, cuML) computations.


In [ ]:
import psutil
import GPUtil

# Function to get memory and CPU usage
def get_memory_and_cpu_usage():
    memory = psutil.virtual_memory()
    cpu = psutil.cpu_percent(interval=1)
    return {
        "cpu_usage_percent": cpu,
        "memory_usage_mb": memory.used / (1024 ** 2),  # Convert bytes to MB
        "memory_total_mb": memory.total / (1024 ** 2),
    }

# Function to get GPU utilization and memory usage
def get_gpu_utilization():
    gpus = GPUtil.getGPUs()
    if not gpus:
        return {"gpu_usage_percent": 0, "gpu_memory_usage_mb": 0, "gpu_memory_total_mb": 0}

    gpu = gpus[0]  # Assuming a single GPU
    return {
        "gpu_usage_percent": gpu.load * 100,  # Convert to percentage
        "gpu_memory_usage_mb": gpu.memoryUsed,
        "gpu_memory_total_mb": gpu.memoryTotal,
    }

## Run benchmarking

In [ ]:
import time
from cuml.manifold import TSNE as TSNE_GPU
from sklearn.manifold import TSNE as TSNE_CPU
from cuml.manifold import UMAP as UMAP_GPU
from umap import UMAP as UMAP_CPU
from cuml.decomposition import PCA as PCA_GPU
from sklearn.decomposition import PCA as PCA_CPU
from cuml.linear_model import LogisticRegression as LogisticRegression_GPU
from sklearn.linear_model import LogisticRegression as LogisticRegression_CPU
from cuml.preprocessing import MinMaxScaler as MinMaxScaler_GPU
from sklearn.preprocessing import MinMaxScaler as MinMaxScaler_CPU
import cudf


# t-SNE analysis (CPU & GPU)
def tsne_analysis(genotype_cpu, genotype_gpu):
    results = {}

    # CPU t-SNE
    start_time = time.time()
    tsne_cpu = TSNE_CPU(n_components=2, random_state=42)
    tsne_cpu.fit_transform(genotype_cpu)
    results["cpu_tsne"] = time.time() - start_time
    # results["cpu_memory_tsne"] = get_memory_and_cpu_usage()

    # GPU t-SNE
    start_time = time.time()
    tsne_gpu = TSNE_GPU(n_components=2, random_state=42)
    tsne_gpu.fit_transform(genotype_gpu)
    results["gpu_tsne"] = time.time() - start_time
    # results["gpu_memory_tsne"] = get_gpu_utilization()

    return results

# UMAP analysis (CPU & GPU)
def umap_analysis(genotype_cpu, genotype_gpu):
    results = {}

    # CPU UMAP
    start_time = time.time()
    umap_cpu = UMAP_CPU(n_components=2, random_state=42)
    umap_cpu.fit_transform(genotype_cpu)
    results["cpu_umap"] = time.time() - start_time
    # results["cpu_memory_umap"] = get_memory_and_cpu_usage()

    # GPU UMAP
    start_time = time.time()
    umap_gpu = UMAP_GPU(n_neighbors=15, n_components=2, random_state=42)
    umap_gpu.fit_transform(genotype_gpu)
    results["gpu_umap"] = time.time() - start_time
    # results["gpu_memory_umap"] = get_gpu_utilization()

    return results

# PCA analysis (CPU & GPU)
def pca_analysis(genotype_cpu, genotype_gpu):
    results = {}

    # CPU PCA
    start_time = time.time()
    pca_cpu = PCA_CPU(n_components=10)
    pca_cpu.fit_transform(genotype_cpu)
    results["cpu_pca"] = time.time() - start_time
    # results["cpu_memory_pca"] = get_memory_and_cpu_usage()

    # GPU PCA
    start_time = time.time()
    pca_gpu = PCA_GPU(n_components=10)
    pca_gpu.fit_transform(genotype_gpu)
    results["gpu_pca"] = time.time() - start_time
    # results["gpu_memory_pca"] = get_gpu_utilization()

    return results

# GWAS analysis (CPU & GPU)
def gwas_analysis(genotype_cpu, genotype_gpu, phenotype_cpu, phenotype_gpu):
    results = {}

    # CPU GWAS
    start_time = time.time()
    scaler_cpu = MinMaxScaler_CPU()
    genotype_scaled_cpu = scaler_cpu.fit_transform(genotype_cpu)
    log_reg_cpu = LogisticRegression_CPU(penalty='l2', max_iter=10000)
    log_reg_cpu.fit(genotype_scaled_cpu, phenotype_cpu)
    results["cpu_gwas"] = time.time() - start_time
    # results["cpu_memory_gwas"] = get_memory_and_cpu_usage()

    # GPU GWAS
    start_time = time.time()
    scaler_gpu = MinMaxScaler_GPU()
    genotype_scaled_gpu = scaler_gpu.fit_transform(genotype_gpu)
    log_reg_gpu = LogisticRegression_GPU(penalty='l2', max_iter=10000)
    log_reg_gpu.fit(genotype_scaled_gpu, phenotype_gpu)
    results["gpu_gwas"] = time.time() - start_time
    # results["gpu_memory_gwas"] = get_gpu_utilization()

    return results

# Central benchmarking function
def benchmark_methods(genotype_matrix_cpu, phenotype_data_cpu, genotype_matrix_gpu, phenotype_data_gpu, method):
    all_results = {}

    if method == "tsne":
        all_results[method] = tsne_analysis(genotype_matrix_cpu, genotype_matrix_gpu)
    elif method == "umap":
        all_results[method] = umap_analysis(genotype_matrix_cpu, genotype_matrix_gpu)
    elif method == "pca":
        all_results[method] = pca_analysis(genotype_matrix_cpu, genotype_matrix_gpu)
    elif method == "gwas":
        all_results[method] = gwas_analysis(genotype_matrix_cpu, genotype_matrix_gpu, phenotype_data_cpu, phenotype_data_gpu)
    
    return all_results


In [ ]:
%%time
n_samples = [1000, 5000]
n_snps = [1000, 5000, 10000, 50000, 100000]

# List of methods to benchmark
methods_to_benchmark = ["tsne", "umap", "pca", "gwas"]

# Store results for each dataset size
benchmarking_results = {}

for sample in n_samples:
    # Initialize the inner dictionary for each sample size
    if sample not in benchmarking_results:
        benchmarking_results[sample] = {}
    for snp in n_snps:
        print(f"############ Running benchmark for dataset with {sample} samples and {snp} SNPs... ############")
        # Generate the synthetic data for the current size
        genotype_matrix_cpu, phenotype_data_cpu, genotype_matrix_gpu, phenotype_data_gpu = generate_synthetic_data(sample, snp)
        
        # Initialize dictionary for each snp size if not done already
        if snp not in benchmarking_results[sample]:
            benchmarking_results[sample][snp] = {}
        
        for method in methods_to_benchmark:
            print("size of matrix: ", genotype_matrix_cpu.nbytes / (1024 * 1024))
            mem_free, mem_total = cp.cuda.Device().mem_info  # Get free and total memory
            mem_used = (mem_total - mem_free) / (1024 * 1024)
            print("free mem used before clearing:", mem_free/(1024*1024))
            # Clear GPU memory after PCA operation
            rmm.reinitialize()
            cp.get_default_memory_pool().free_all_blocks()
            mem_free, mem_total = cp.cuda.Device().mem_info  # Get free and total memory
            print("free mem used after clearing:", mem_free/(1024*1024))
            print(method)
            # Call the benchmark_methods function with the specified methods and store the results
            try:
                # Merge the result from the method into the current snp dictionary
                method_results = benchmark_methods(genotype_matrix_cpu, phenotype_data_cpu, genotype_matrix_gpu, phenotype_data_gpu, method)
                benchmarking_results[sample][snp].update(method_results)
            except Exception as e:
                print(f"Error during benchmarking for {sample} samples and {snp} SNPs and {method}: {str(e)}")
                benchmarking_results[sample][snp][method] = str(e)
        
        # Store the size of the matrix in MB
        benchmarking_results[sample][snp]["size_in_mb"] = genotype_matrix_cpu.nbytes / (1024 * 1024)  # Convert bytes to MB

In [ ]:
benchmarking_results

In [ ]:
# Reformat the data into the desired format with error handling for failed operations
formatted_data = []

for samples, snps_dict in benchmarking_results.items():
    for snps, methods in snps_dict.items():
        row = {
            "samples": samples,
            "snps": snps,
            "size_in_mb": methods.get('size_in_mb', None),
            "cpu_tsne": methods.get('tsne', {}).get('cpu_tsne', "error"),
            "gpu_tsne": methods.get('tsne', {}).get('gpu_tsne', "error"),
            "cpu_umap": methods.get('umap', {}).get('cpu_umap', "error"),
            "gpu_umap": methods.get('umap', {}).get('gpu_umap', "error"),
            "cpu_pca": methods['pca'].get('cpu_pca', "error") if isinstance(methods.get('pca'), dict) else "error",
            "gpu_pca": methods['pca'].get('gpu_pca', "error") if isinstance(methods.get('pca'), dict) else "error",
            "cpu_gwas": methods.get('gwas', {}).get('cpu_gwas', "error"),
            "gpu_gwas": methods.get('gwas', {}).get('gpu_gwas', "error")
        }
        formatted_data.append(row)

# Create a DataFrame from the formatted data
df_full_formatted = pd.DataFrame(formatted_data)
df_full_formatted

In [ ]:
import matplotlib.pyplot as plt

# Convert 'error' values to None (to handle plotting)
for key in ['cpu_pca', 'gpu_pca']:
    df_full_formatted[key] = [float(x) if x != 'error' else None for x in df_full_formatted[key]]

# Create subplots for the four methods: t-SNE, UMAP, PCA, GWAS
fig, axs = plt.subplots(4, 2, figsize=(12, 16))

# Plot t-SNE
axs[0, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['cpu_tsne'][:5], label='CPU t-SNE', marker='o')
axs[0, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['gpu_tsne'][:5], label='GPU t-SNE', marker='o')
axs[0, 0].set_title('t-SNE (1000 samples)')
axs[0, 0].set_xlabel('SNPs')
axs[0, 0].set_ylabel('Run Time (s)')
axs[0, 0].legend()

axs[0, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['cpu_tsne'][5:], label='CPU t-SNE', marker='o')
axs[0, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['gpu_tsne'][5:], label='GPU t-SNE', marker='o')
axs[0, 1].set_title('t-SNE (5000 samples)')
axs[0, 1].set_xlabel('SNPs')
axs[0, 1].set_ylabel('Run Time (s)')
axs[0, 1].legend()

# Plot UMAP
axs[1, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['cpu_umap'][:5], label='CPU UMAP', marker='o')
axs[1, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['gpu_umap'][:5], label='GPU UMAP', marker='o')
axs[1, 0].set_title('UMAP (1000 samples)')
axs[1, 0].set_xlabel('SNPs')
axs[1, 0].set_ylabel('Run Time (s)')
axs[1, 0].legend()

axs[1, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['cpu_umap'][5:], label='CPU UMAP', marker='o')
axs[1, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['gpu_umap'][5:], label='GPU UMAP', marker='o')
axs[1, 1].set_title('UMAP (5000 samples)')
axs[1, 1].set_xlabel('SNPs')
axs[1, 1].set_ylabel('Run Time (s)')
axs[1, 1].legend()

# Plot PCA
axs[2, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['cpu_pca'][:5], label='CPU PCA', marker='o')
axs[2, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['gpu_pca'][:5], label='GPU PCA', marker='o')
axs[2, 0].set_title('PCA (1000 samples)')
axs[2, 0].set_xlabel('SNPs')
axs[2, 0].set_ylabel('Run Time (s)')
axs[2, 0].legend()

axs[2, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['cpu_pca'][5:], label='CPU PCA', marker='o')
axs[2, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['gpu_pca'][5:], label='GPU PCA', marker='o')
axs[2, 1].set_title('PCA (5000 samples)')
axs[2, 1].set_xlabel('SNPs')
axs[2, 1].set_ylabel('Run Time (s)')
axs[2, 1].legend()

# Plot GWAS
axs[3, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['cpu_gwas'][:5], label='CPU GWAS', marker='o')
axs[3, 0].plot(df_full_formatted['snps'][:5], df_full_formatted['gpu_gwas'][:5], label='GPU GWAS', marker='o')
axs[3, 0].set_title('GWAS (1000 samples)')
axs[3, 0].set_xlabel('SNPs')
axs[3, 0].set_ylabel('Run Time (s)')
axs[3, 0].legend()

axs[3, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['cpu_gwas'][5:], label='CPU GWAS', marker='o')
axs[3, 1].plot(df_full_formatted['snps'][5:], df_full_formatted['gpu_gwas'][5:], label='GPU GWAS', marker='o')
axs[3, 1].set_title('GWAS (5000 samples)')
axs[3, 1].set_xlabel('SNPs')
axs[3, 1].set_ylabel('Run Time (s)')
axs[3, 1].legend()

# Adjust layout and display
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Colors for Intel and Nvidia
intel_blue = '#0071C5'
nvidia_green = '#76B900'

# Bar width for the bars
bar_width = 0.35
# Labels for x-axis ticks
snps_labels_1000 = [str(x) for x in df_full_formatted['snps'][:5]]
snps_labels_5000 = [str(x) for x in df_full_formatted['snps'][5:]]

def add_annotations(ax, rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        if height is not None:
            ax.annotate(f'{height:.2f}',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

def plot_and_save(cpu_df_full_formatted, gpu_df_full_formatted, title, filename, snps_labels_1000, snps_labels_5000):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    
    # Filter out None values for 1000 samples
    cpu_df_full_formatted_1000 = [d for d in cpu_df_full_formatted[:5] if d is not None]
    gpu_df_full_formatted_1000 = [d for d in gpu_df_full_formatted[:5] if d is not None]
    valid_snps_1000 = [s for s, d in zip(snps_labels_1000, cpu_df_full_formatted[:5]) if d is not None]

    # Plot for 1000 samples
    x = np.arange(len(valid_snps_1000))
    cpu_rects_1000 = axs[0].bar(x - bar_width/2, cpu_df_full_formatted_1000, bar_width, label='CPU', color=intel_blue)
    gpu_rects_1000 = axs[0].bar(x + bar_width/2, gpu_df_full_formatted_1000, bar_width, label='GPU', color=nvidia_green)
    axs[0].set_title(f'{title} (1000 samples)')
    axs[0].set_xlabel('SNPs')
    axs[0].set_ylabel('Run Time (s)')
    axs[0].set_xticks(x)
    axs[0].set_xticklabels(valid_snps_1000)
    axs[0].legend()
    add_annotations(axs[0], cpu_rects_1000)
    add_annotations(axs[0], gpu_rects_1000)

    # Filter out None values for 5000 samples
    cpu_df_full_formatted_5000 = [d for d in cpu_df_full_formatted[5:] if d is not None]
    gpu_df_full_formatted_5000 = [d for d in gpu_df_full_formatted[5:] if d is not None]
    valid_snps_5000 = [s for s, d in zip(snps_labels_5000, cpu_df_full_formatted[5:]) if d is not None]

    # Plot for 5000 samples
    x = np.arange(len(valid_snps_5000))
    cpu_rects_5000 = axs[1].bar(x - bar_width/2, cpu_df_full_formatted_5000, bar_width, label='CPU', color=intel_blue)
    gpu_rects_5000 = axs[1].bar(x + bar_width/2, gpu_df_full_formatted_5000, bar_width, label='GPU', color=nvidia_green)
    axs[1].set_title(f'{title} (5000 samples)')
    axs[1].set_xlabel('SNPs')
    axs[1].set_ylabel('Run Time (s)')
    axs[1].set_xticks(x)
    axs[1].set_xticklabels(valid_snps_5000)
    axs[1].legend()
    add_annotations(axs[1], cpu_rects_5000)
    add_annotations(axs[1], gpu_rects_5000)

    # Adjust layout and save the figure
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# Save each method as a separate PNG file
plot_and_save(df_full_formatted['cpu_tsne'], df_full_formatted['gpu_tsne'], 't-SNE', 'tsne.png', snps_labels_1000, snps_labels_5000)
plot_and_save(df_full_formatted['cpu_umap'], df_full_formatted['gpu_umap'], 'UMAP', 'umap.png', snps_labels_1000, snps_labels_5000)
plot_and_save(df_full_formatted['cpu_pca'], df_full_formatted['gpu_pca'], 'PCA', 'pca.png', snps_labels_1000, snps_labels_5000)
plot_and_save(df_full_formatted['cpu_gwas'], df_full_formatted['gpu_gwas'], 'GWAS', 'gwas.png', snps_labels_1000, snps_labels_5000)
